<a href="https://colab.research.google.com/github/Jtranod98/Amazon_Vine_Analysis/blob/main/Amazon_Reviews_ETL.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import os
# Find the latest version of spark 3.0 from http://www.apache.org/dist/spark/ and enter as the spark version
# For example:
# spark_version = 'spark-3.0.3'
spark_version = 'spark-3.0.3'
os.environ['SPARK_VERSION']=spark_version

# Install Spark and Java
!apt-get update
!apt-get install openjdk-11-jdk-headless -qq > /dev/null
!wget -q http://www.apache.org/dist/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop2.7.tgz
!tar xf $SPARK_VERSION-bin-hadoop2.7.tgz
!pip install -q findspark

# Set Environment Variables
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop2.7"

# Start a SparkSession
import findspark
findspark.init()

Get:1 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Get:2 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease [3,626 B]
Get:3 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease [1,581 B]
Get:4 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease [15.9 kB]
Hit:5 http://archive.ubuntu.com/ubuntu bionic InRelease
Get:6 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Ign:7 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Hit:8 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Get:9 http://security.ubuntu.com/ubuntu bionic-security/universe amd64 Packages [1,527 kB]
Hit:10 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Get:11 http://security.ubuntu.com/ubuntu bionic-security/main amd64 Packages [2,901 kB]
Get:12 http://archive.ubuntu.com/ubuntu bionic-backports InRelease

In [ ]:
# Download the Postgres driver that will allow Spark to interact with Postgres.
!wget https://jdbc.postgresql.org/download/postgresql-42.2.16.jar

--2022-07-23 03:07:35--  https://jdbc.postgresql.org/download/postgresql-42.2.16.jar
Resolving jdbc.postgresql.org (jdbc.postgresql.org)... 72.32.157.228, 2001:4800:3e1:1::228
Connecting to jdbc.postgresql.org (jdbc.postgresql.org)|72.32.157.228|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1002883 (979K) [application/java-archive]
Saving to: ‘postgresql-42.2.16.jar’

postgresql-42.2.16. 100%[===================>] 979.38K  5.76MB/s    in 0.2s    

2022-07-23 03:07:35 (5.76 MB/s) - ‘postgresql-42.2.16.jar’ saved [1002883/1002883]



In [ ]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("M16-Amazon-Challenge").config("spark.driver.extraClassPath","/content/postgresql-42.2.16.jar").getOrCreate()

### Load Amazon Data into Spark DataFrame

In [ ]:
from pyspark import SparkFiles
url = "https://s3.amazonaws.com/amazon-reviews-pds/tsv/amazon_reviews_us_Wireless_v1_00.tsv.gz"
spark.sparkContext.addFile(url)
df = spark.read.option("encoding", "UTF-8").csv(SparkFiles.get(""), sep="\t", header=True, inferSchema=True)
df.show()

+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|marketplace|customer_id|     review_id|product_id|product_parent|       product_title|product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|     review_headline|         review_body|review_date|
+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|         US|   16414143|R3W4P9UBGNGH1U|B00YL0EKWE|     852431543|LG G4 Case Hard T...|        Wireless|          2|            1|          3|   N|                Y|Looks good, funct...|2 issues  -  Once...| 2015-08-31|
|         US|   50800750|R15V54KBMTQWAY|B00XK95RPQ|     516894650|Selfie Stick Fibl...|        Wireless|          4|    

### Create DataFrames to match tables

In [ ]:
from pyspark.sql.functions import to_date
# Read in the Review dataset as a DataFrame


In [ ]:
# Create the customers_table DataFrame
customers_df = df.groupby("customer_id").agg({"customer_id":"count"}).withColumnRenamed("count(customer_id)", "customer_count")

customers_df.show()

+-----------+--------------+
|customer_id|customer_count|
+-----------+--------------+
|   46909180|             6|
|   42560427|             7|
|   43789873|             3|
|   22037526|             2|
|   34220092|             2|
|   42801586|             1|
|    9565734|             2|
|   15829398|             1|
|   38247118|             1|
|   32478248|             2|
|   48114630|             1|
|   23085063|             1|
|   32787070|             3|
|   43515569|             1|
|    4919528|             2|
|    5088547|             2|
|   41852407|             3|
|   49703087|             1|
|   12713799|             1|
|   36728141|             8|
+-----------+--------------+
only showing top 20 rows



In [ ]:
# Create the products_table DataFrame and drop duplicates. 
products_df = df.select(["product_id", "product_title"]).drop_duplicates(['product_id'])
products_df.show()

+----------+--------------------+
|product_id|       product_title|
+----------+--------------------+
|1060104342|Premium Asus MeMO...|
|159775045X|National Geograph...|
|5891090414|           STAND-N02|
|6175001532|Colorful Abstract...|
|9572548964|Pink Flower Purpl...|
|9713000463|Rapid Charger KIT...|
|9838427853|PowerBear® Samsun...|
|9861033181|Purple Flower Blu...|
|9983781239|PREMIUM USB Adapt...|
|9984225437|Sanyo MM-8300 Sta...|
|B00000J0IV|Magellan Leather ...|
|B00000J1PX|Cobra MAVOX Voice...|
|B00003OPEV|Garmin eMap Delux...|
|B0000520T4|Ericsson 650 mAh ...|
|B00005I9QK|Aquapac Waterproo...|
|B00005QT35|Garmin GPS 45 Per...|
|B00006JP9X|AMERICAN INTERNAT...|
|B00007EDL7|Sony DCC-E34CP Po...|
|B00008E1OH|Samsung Leather C...|
|B00008RCQS|Virgin Mobile Pay...|
+----------+--------------------+
only showing top 20 rows



In [ ]:
# Create the review_id_table DataFrame. 
# Convert the 'review_date' column to a date datatype with to_date("review_date", 'yyyy-MM-dd').alias("review_dat
review_id_df = df.select(["review_id", "customer_id", "product_id", "product_parent", to_date("review_date", 'yyyy-MM-dd').alias("review_date")])
review_id_df = review_id_df.drop_duplicates(["review_id"])
review_id_df.show()

+--------------+-----------+----------+--------------+-----------+
|     review_id|customer_id|product_id|product_parent|review_date|
+--------------+-----------+----------+--------------+-----------+
|R1002OCUHG9RB2|   12054485|B00822KA0S|     678284285| 2013-11-13|
|R1009FXGIMJ5A8|   13899950|B001OD2OAQ|     177396772| 2011-09-08|
|R100B5W7AF3U5Y|     513891|B00NIEPGFE|     906325961| 2015-04-07|
|R100L00EV9JBCG|    6910893|B005XEQN3Q|     161673841| 2014-03-22|
|R100P3GPMNRH4B|   38640325|B00BRN9IRA|     696253827| 2014-04-17|
|R100P4C2W5ZH6T|   41144872|B00YG6H4XA|     107979040| 2015-08-29|
|R100PCW6NYIFPQ|   16476751|B003OQUKT8|     312176524| 2011-03-19|
|R100S2HJBXXWGL|   23474977|B008HQQ6B6|     452589858| 2013-01-17|
|R100ZL6728NAXN|   14564336|B000U9WVW6|     751208272| 2009-08-31|
|R1010XQ6K0JQP1|     443291|B00MATHF16|     132751708| 2015-03-05|
|R1012SVT1P3771|   20060551|B00MD796LW|     829643552| 2015-01-04|
|R1014Z60YEPUM7|   28068239|B004RBL51Q|     165875437| 2012-01

In [ ]:
# Create the vine_table. DataFrame
from pyspark.sql.types import IntegerType
df = df.withColumn("star_rating", df["star_rating"].cast(IntegerType()))
vine_df = df.select(["review_id", "star_rating", "helpful_votes", "total_votes", "vine", "verified_purchase"]).drop_duplicates(["review_id"])

vine_df.show()

+--------------+-----------+-------------+-----------+----+-----------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|
+--------------+-----------+-------------+-----------+----+-----------------+
|R1002OCUHG9RB2|          4|            0|          0|   N|                Y|
|R1009FXGIMJ5A8|          1|            0|          2|   N|                Y|
|R100B5W7AF3U5Y|          5|            0|          2|   N|                Y|
|R100L00EV9JBCG|          5|            0|          0|   N|                Y|
|R100P3GPMNRH4B|          5|            0|          0|   N|                Y|
|R100P4C2W5ZH6T|          5|            0|          0|   N|                Y|
|R100PCW6NYIFPQ|          5|            0|          0|   N|                Y|
|R100S2HJBXXWGL|          2|            0|          0|   N|                Y|
|R100ZL6728NAXN|          3|            0|          1|   N|                N|
|R1010XQ6K0JQP1|          3|            0|          0|   N|     

### Connect to the AWS RDS instance and write each DataFrame to its table. 

In [ ]:
# Store environmental variable
from getpass import getpass
password = getpass('Enter database password')
# Configure settings for RDS
mode = "append"

jdbc_url="jdbc:postgresql://<endpoint>:5432/amazon_reviews_db"
config = {"user":"postgres",
          "password": password,
          "driver":"org.postgresql.Driver"}


Enter database password··········


In [ ]:
# Write review_id_df to table in RDS
review_id_df.write.jdbc(url=jdbc_url, table='review_id_table', mode=mode, properties=config)

In [ ]:
# Write products_df to table in RDS
# about 3 min
products_df.write.jdbc(url=jdbc_url, table='products_table', mode=mode, properties=config)

In [ ]:
# Write customers_df to table in RDS
# 5 min 14 s
customers_df.write.jdbc(url=jdbc_url, table='customers_table', mode=mode, properties=config)

In [ ]:
# Write vine_df to table in RDS
# 11 minutes
vine_df.write.jdbc(url=jdbc_url, table='vine_table', mode=mode, properties=config)